In [1]:
import numpy as np
from elasticsearch import helpers, Elasticsearch
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
from spellchecker import SpellChecker

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()


{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'fpFxjw6cRjyGQOMl6ThLdw',
 'version': {'number': '7.5.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'e9ccaed468e2fac2275a3761849cbee64b39519f',
  'build_date': '2019-11-26T01:06:52.518245Z',
  'build_snapshot': False,
  'lucene_version': '8.3.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
df_ingred = pd.read_pickle('/Users/s0c02nj/Downloads/df_ingred')

In [4]:
df_ingred.shape

(31033, 2)

In [6]:
#df_ingred_gv = pd.read_pickle('/Users/s0c02nj/Desktop/Item_search_Std_Ref/great_value_and')

In [ ]:
# ingred_gv = list(set(df_ingred_gv['Ingredients_cln']))
# ingred_gv = list(filter(None, ingred_gv))

In [ ]:
#df_ingred_non_gv = pd.read_pickle('/Users/s0c02nj/Desktop/Item_search_Std_Ref/non_great_value_and')

In [ ]:
# ingred_non_gv = list(set(df_ingred_non_gv['Ingredients_cln']))
# ingred_non_gv = list(filter(None, ingred_non_gv))

In [ ]:
#ingred_total_rem = ingred_gv + ingred_non_gv

In [ ]:
#df_ingred = df_ingred[~df_ingred['Ingredients_cln'].isin(ingred_total_rem)]

In [ ]:
#df_ingred.shape

In [5]:
data_df = pd.read_pickle('/Users/s0c02nj/Downloads/data_df')

In [ ]:
index='std_ref'

In [ ]:
document = data_df[['upc_nbr','qarth_product_name','signing_desc']]

In [ ]:
def doc_generator(data,index):
    df_iter=data.iterrows()
    for index,document in df_iter:
        yield{
            "_index": 'std_ref',
           "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration
    
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(document,index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

In [ ]:
def SEARCH(text,index,field):
    res=es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"}}}},request_timeout=30)
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])



In [ ]:
def query_generator(df):
    while True:
        yield (df['Ingredients_cln'])

In [ ]:
query_generator(df_ingred)

In [ ]:
import concurrent.futures

In [ ]:
start = time.time()
queries=[]
with concurrent.futures.ThreadPoolExecutor() as executor:
    queries={executor.submit(SEARCH, query,index,'qarth_product_name'): query for query in df_ingred.Ingredients_cln}
print(time.time()-start)



In [ ]:
#queries

In [ ]:
out = [x.result() for x in queries]

In [ ]:
len(out)

In [ ]:
df_ingred.shape

In [ ]:
def get_std_ref_match(data_current):
    
    #UPC to match
    ingred_to_match = []
    #Original
    ingred_org = []
    #Matched UPC
    upc_matched = []
    #Qarth Desc
    qarth_desc = []
    #Signing Desc
    sign_desc = []
    #Score
    score_qarth = []
    
    #Reindex
    data_current.index = range(0,len(data_current))
    
    
    for i in tqdm(range(0,len(data_current))):

            #indexing
            #data_current.index = range(0,len(data_current))

            word = data_current['Ingredients_cln'].iloc[i]
            ingred = data_current['Ingredients'].iloc[i]

            #Getting the search result
            temp_qarth = out[i]
            val = len(temp_qarth)

            if val ==0:
                #clean_ingred
                ingred_to_match.append(word)

                #org_ingred
                ingred_org.append(ingred)

                #Matched UPC of top3 WM_SR items
                upc_matched.append(0)

                #Signing desc of the matched
                sign_desc.append('NA')

                #Qarth_product_name of the matched
                qarth_desc.append('NA')

                #Score
                score_qarth.append('NA')

            else:
                for j in np.arange(val):

                    #clean_ingred
                    ingred_to_match.append(word)

                    #org_ingred
                    ingred_org.append(ingred)

                    #Creating temp
                    temp2 = temp_qarth[j]

                    #Separting the tiple
                    doc_txt = temp2[0]
                    doc_scr = temp2[1]


                    #Matched UPC of top3 WM_SR items
                    upc_matched.append(doc_txt['upc_nbr'])

                    #Signing desc of the matched
                    sign_desc.append(doc_txt['signing_desc'])

                    #Qarth_product_name of the matched
                    qarth_desc.append(doc_txt['qarth_product_name'])

                    #Score
                    score_qarth.append(doc_scr)


    #Converting to a Dataframe   
    df_result = pd.DataFrame()
    df_result['Ingredients_cln'] = ingred_to_match
    df_result['Ingredients_org'] = ingred_org
    df_result['UPC_match'] = upc_matched
    df_result['Signing_Desc_Match'] = sign_desc
    df_result['Qarth_Desc_Match'] = qarth_desc
    df_result['list_score_qarth'] = score_qarth

    return df_result



In [ ]:
df_result_or = get_std_ref_match(df_ingred)

In [ ]:
df_result_or.isna().sum()

In [ ]:
df_result_or[df_result_or['UPC_match'] != 0]['Ingredients_cln'].unique().shape

In [ ]:
df_ingred['Ingredients_cln'].unique().shape

In [ ]:
df_result_or.head()

In [ ]:
df_result_or = df_result_or[df_result_or['UPC_match'] != 0]

In [ ]:
df_result_or.to_csv('/Users/s0c02nj/Desktop/Item_search_Std_Ref/result_or.csv',index = False)

In [ ]:
df_result_or.to_pickle('/Users/s0c02nj/Desktop/Item_search_Std_Ref/result_or')